### laser_visualisation_Jupyter-ros

Visualisation of the depth camera as a laser sensor in a 3D setting using jupyter ros.

In [ ]:
import sys; sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
from os import environ
import rospy
from geometry_msgs.msg import Pose, PoseStamped, PoseWithCovarianceStamped
environ['ROS_MASTER_URI'] = "http://localhost:11311/" #Ordi personnel

In [ ]:
# Créer et démarrer un nouveau noeud
# (en) Create and start a new node
rospy.init_node('visualisation', anonymous=True)

In [ ]:
# AMCL_pose callback (pour la visualisation de la pose)
pose_msg = Pose()
def amcl_callback(msg):
    global pose_msg
    pose_msg = msg.pose.pose
    pose = PoseStamped()

    pose.header.seq = 1
    pose.header.stamp = rospy.Time.now()
    pose.header.frame_id = "map"

    pose.pose.position.x = pose_msg.position.x
    pose.pose.position.y = pose_msg.position.y
    pose.pose.position.z = 0.0

    pose.pose.orientation.x = pose_msg.orientation.x
    pose.pose.orientation.y = pose_msg.orientation.y
    pose.pose.orientation.z = pose_msg.orientation.z
    pose.pose.orientation.w = pose_msg.orientation.w

    pose_pub.publish(pose)

pose_pub = rospy.Publisher('/mobile_manip/pose', PoseStamped, queue_size=1)
amcl_sub = rospy.Subscriber('/amcl_pose', PoseWithCovarianceStamped, amcl_callback)

In [ ]:
from jupyros import ros3d
from ipywidgets import Layout
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090")
tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')

laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client)
pose_view = ros3d.Pose(topic="/mobile_manip/pose", ros=rc, tf_client=tf_client)
g = ros3d.GridModel()
v.objects = [laser_view, map_view, pose_view]
v.layout = Layout(border="3px solid black", width="1200px", height="500px")
v